In [ ]:
import numpy as np

def sabr_mc_log_euler_paths(F0, alpha0, beta, nu, rho, T, num_steps, num_paths):
    """
    Generates paths for the SABR model using the log-Euler discretization scheme.

    Parameters:
    F0 (float): Initial forward price.
    alpha0 (float): Initial volatility.
    beta (float): Elasticity parameter (0 <= beta <= 1).
    nu (float): Volatility-of-volatility parameter (nu > 0).
    rho (float): Correlation parameter (-1 <= rho <= 1).
    T (float): Time to maturity (in years).
    num_steps (int): Number of time steps for the simulation.
    num_paths (int): Number of simulation paths to generate.

    Returns:
    tuple: (F_paths, alpha_paths)
        F_paths (np.ndarray): Simulated forward price paths.
                              Shape: (num_steps + 1, num_paths).
        alpha_paths (np.ndarray): Simulated volatility paths.
                                  Shape: (num_steps + 1, num_paths).
    """
    dt = T / num_steps

    # Initialize arrays to store paths
    F_paths = np.zeros((num_steps + 1, num_paths))
    alpha_paths = np.zeros((num_steps + 1, num_paths))

    F_paths[0, :] = F0
    alpha_paths[0, :] = alpha0

    # Pre-generate all random numbers
    Z1 = np.random.standard_normal((num_steps, num_paths))
    Z2 = np.random.standard_normal((num_steps, num_paths))

    sqrt_dt = np.sqrt(dt)

    for i in range(num_steps):
        # Current state
        F_i = F_paths[i, :]
        alpha_i = alpha_paths[i, :]

        # Ensure alpha remains positive (log-Euler for alpha naturally does this)
        # alpha_i = np.maximum(alpha_i, 1e-10) # Absorption barrier if needed, but log-Euler should prevent negatives

        # Volatility process update (Log-Euler)
        alpha_paths[i+1, :] = alpha_i * np.exp(nu * Z1[i, :] * sqrt_dt - 0.5 * nu**2 * dt)
        
        # Handle F_i^(beta-1) term
        # This term can be problematic if F_i is zero or negative.
        # However, log-Euler for F should keep F_i > 0 if F0 > 0.
        # For normalized S_bar_t starting at 1, this should be fine.
        if beta == 1.0:
            F_beta_minus_1 = 1.0
        elif beta == 0.0:
            # Requires F_i > 0. If F_i can be very close to zero, this might be unstable.
            # For S_bar_t, F_i is S_bar_i, which should be > 0.
            F_beta_minus_1 = 1.0 / F_i 
        else:
            F_beta_minus_1 = F_i**(beta - 1.0)

        # Correlated Wiener increment for F
        dW1_i = (rho * Z1[i, :] + np.sqrt(1 - rho**2) * Z2[i, :]) * sqrt_dt
        
        # Forward price process update (Log-Euler)
        # d(ln F_t) = alpha_t * F_t^(beta-1) dW1_t - 0.5 * (alpha_t * F_t^(beta-1))^2 dt
        exponent_term = alpha_i * F_beta_minus_1 * dW1_i - 0.5 * (alpha_i * F_beta_minus_1)**2 * dt
        F_paths[i+1, :] = F_i * np.exp(exponent_term)
        
        # Ensure F_paths remain positive (log-Euler for F naturally does this)
        F_paths[i+1, :] = np.maximum(F_paths[i+1, :], 1.e-50) # Absorption if needed

        # make sure the averaged F is F0 otherwise scale it
        scale= F0/np.mean(F_paths[i+1, :])
        F_paths[i+1,:] *=scale

    # scale= F0/np.mean(F_paths[-1, :])
    # F_paths[-1,:] *=scale
    return F_paths, alpha_paths

def simulate_displaced_sabr(S0, delta, alpha0, beta, nu, rho, T, num_steps, num_paths):
    """
    Simulates paths for the displaced SABR model using normalization.
    """
    if S0 + delta <= 0:
        raise ValueError("S0 + delta must be positive for normalization.")

    # Simulate the normalized process S_bar_t, which starts at 1
    # The alpha0, beta, nu, rho parameters are for this normalized process.
    # Typically, the market-quoted alpha0 is for the normalized process or directly usable.
    # The beta for S_bar_t is the same beta for (S_t + delta).
    
    S_bar_0 = 1.0
    S_bar_0 = S0+delta
    # Assuming alpha0 is the volatility parameter for the S_bar process.
    # If alpha0 was for S_t+delta, it might need scaling: alpha0_bar = alpha0 * (S0+delta)**(beta-1)
    # However, SABR parameters are often quoted such that alpha0 is directly usable as alpha0_bar.
    
    S_bar_paths, alpha_bar_paths = sabr_mc_log_euler_paths(
        F0=S_bar_0, 
        alpha0=alpha0, # Use the provided alpha0 directly for S_bar
        beta=beta, 
        nu=nu, 
        rho=rho, 
        T=T, 
        num_steps=num_steps, 
        num_paths=num_paths
    )

    # Recover S_t paths from S_bar_t paths
    # S_t = S_bar_t * (S0 + delta) - delta
    #S_t_paths = S_bar_paths * (S0 + delta) - delta
    S_t_paths = S_bar_paths  - delta
    
    # alpha_bar_paths are the paths for the volatility of S_bar_t.
    # If paths for alpha_t (vol of S_t+delta) are needed and alpha_bar was scaled,
    # then alpha_t_paths = alpha_bar_paths / (S0+delta)**(beta-1)
    # But usually alpha_bar_paths are what's relevant if parameters are for normalized dynamics.
    
    return S_t_paths, alpha_bar_paths # Returning alpha_bar_paths as the vol paths

def sabr_simulation_regularized(F0, sigma0, beta, nu, rho, T, dt, num_simulations):
    """
    Simulates the SABR model using the regularized Euler scheme from the paper's appendix.

    Args:
        F0 (float): Initial forward price.
        sigma0 (float): Initial volatility.
        beta (float): Beta parameter (elasticity).
        nu (float): Nu parameter (vol-of-vol).
        rho (float): Rho parameter (correlation).
        T (float): Time to maturity.
        dt (float): Time step size.
        num_simulations (int): Number of Monte Carlo simulations.

    Returns:
        numpy.ndarray: Array of simulated final forward prices.
    """
    num_steps = int(T / dt)
    
    # Initialize arrays
    F = np.full(num_simulations, F0)
    sigma = np.full(num_simulations, sigma0)
    
    # Transformation parameters
    X0 = F0**(1 - beta) / (1 - beta)
    B = beta / (2 * (1 - beta))
    epsilon = 1e-6  # Regularization parameter

    X = np.full(num_simulations, X0)
    M = np.full(num_simulations, X0)
    Y_eps = np.zeros(num_simulations)

    # Correlated random numbers
    Z1 = np.random.normal(size=(num_steps, num_simulations))
    Z2 = np.random.normal(size=(num_steps, num_simulations))
    dW = Z1 * np.sqrt(dt)
    dZ = (rho * Z1 + np.sqrt(1 - rho**2) * Z2) * np.sqrt(dt)

    for i in range(num_steps):
        # Update sigma
        sigma = sigma * np.exp(nu * dZ[i] - 0.5 * nu**2 * dt)

        # Update M
        M = M + sigma * dW[i]

        # Update Y_eps
        Y_eps = Y_eps + B * sigma**2 / np.maximum(M - Y_eps, epsilon) * dt
        
        # Update X
        X_prev = X
        X = np.where(X_prev == 0, 0, M - Y_eps)
        X = np.where(Y_eps >= M - epsilon, 0, X)

    # Transform back to F
    F_T = ((1 - beta) * X)**(1 / (1 - beta))
    return F_T

def sabr_simulation_log_euler(F0, sigma0, beta, nu, rho, T, dt, num_simulations):
    """
    Simulates the SABR model using a log-Euler discretization scheme.

    Args:
        F0 (float): Initial forward price.
        sigma0 (float): Initial volatility.
        beta (float): Beta parameter (elasticity).
        nu (float): Nu parameter (vol-of-vol).
        rho (float): Rho parameter (correlation).
        T (float): Time to maturity.
        dt (float): Time step size.
        num_simulations (int): Number of Monte Carlo simulations.

    Returns:
        numpy.ndarray: Array of simulated final forward prices.
    """
    num_steps = int(T / dt)
    
    # Initialize arrays
    F = np.full(num_simulations, F0)
    sigma = np.full(num_simulations, sigma0)

    # Correlated random numbers
    Z1 = np.random.normal(size=(num_steps, num_simulations))
    Z2 = np.random.normal(size=(num_steps, num_simulations))
    dW = Z1 * np.sqrt(dt)
    dZ = (rho * Z1 + np.sqrt(1 - rho**2) * Z2) * np.sqrt(dt)
    
    for i in range(num_steps):
        # Update F using log-Euler
        F = F * np.exp(sigma * F**(beta - 1) * dW[i] - 0.5 * sigma**2 * F**(2 * beta - 2) * dt)
        # Ensure F_paths remain positive (log-Euler for F naturally does this)
        F = np.maximum(F, 1.e-50) # Absorption if needed
        # Update sigma using log-Euler
        sigma = sigma * np.exp(nu * dZ[i] - 0.5 * nu**2 * dt)

    return F


import numpy as np
from scipy.integrate import quad
from scipy.special import iv

#def sabr_implied_vol(K, F=1, T=3, alpha=0.5, beta=0.6, rho=-0.2, nu=0.3):
def sabr_implied_vol(K, T=20, F=1, alpha=0.5, beta=0.6, rho=-0.2, nu=0.3):
    """
    Computes the SABR-implied Black volatility using Hagan's approximation.
    
    Parameters:
        K (float): Strike price
        F (float): Forward price
        T (float): Time to maturity
        alpha (float): SABR volatility parameter
        beta (float): SABR elasticity parameter (0 <= beta <= 1)
        rho (float): SABR correlation parameter (-1 <= rho <= 1)
        nu (float): SABR volatility of volatility parameter

    Returns:
        float: The implied volatility (Black volatility)
    """
    
    K = np.asarray(K, dtype=np.float64)
    eps = 1e-07
    # Determine the ATM condition for each K
    atm = np.abs(F - K) < eps

    # ATM branch
    vol_atm = (alpha / (F**(1-beta))) * (
        1 + (
            ((1-beta)**2 / 24) * (alpha**2 / (F**(2-2*beta))) +
            (rho * beta * nu * alpha) / (4 * (F**(1-beta))) +
            ((2-3*rho**2) / 24) * nu**2
        ) * T
    )

    # Non-ATM branch
    log_fk = np.log(F / K)
    fk_beta = (F * K)**((1-beta)/2)
    z = (nu / alpha) * fk_beta * log_fk
    sqrt_expr = np.sqrt(1 - 2 * rho * z + z**2)
    x_z = np.where(
        np.abs(z) > eps,
        np.log((sqrt_expr + z - rho) / (1 - rho)),
        z - 0.5 * rho * z**2  # Series expansion for small z
    )
    term1 = ((1-beta)**2 / 24) * (alpha**2 / (fk_beta**2))
    term2 = (rho * beta * nu * alpha) / (4 * fk_beta)
    term3 = ((2-3*rho**2) / 24) * nu**2
    vol_nonatm = (alpha / (((F * K)**((1-beta)/2)) * (1 + ((1-beta)**2/24)*(log_fk**2) + ((1-beta)**4/1920)*(log_fk**4)))) \
                 * (z / x_z) * (1 + (term1 + term2 + term3) * T)

    vol = np.where(atm, vol_atm, vol_nonatm)
    # Return a scalar if input was scalar
    if vol.size == 1:
        return float(vol)
    return vol

def sabr_xzz_pdf(K, T=20, F=1, alpha=0.5, beta=0.5, rho=-0.2, nu=0.3):
    """
    Computes the SABR-implied Black volatility using Hagan's approximation.
    
    Parameters:
        K (float): Strike price
        F (float): Forward price
        T (float): Time to maturity
        alpha (float): SABR volatility parameter
        beta (float): SABR elasticity parameter (0 <= beta <= 1)
        rho (float): SABR correlation parameter (-1 <= rho <= 1)
        nu (float): SABR volatility of volatility parameter

    Returns:
        float: The implied volatility (Black volatility)
    """
    # Implementation details for the SABR XZZ model would go here
    r = 0.5/(1-beta)
    def Z(f):
        return f**(1-beta)/alpha/(1-beta)
    def J(z):
        return (1 - 2*rho*nu*z + (nu*z)**2)**0.5
    def X(z):
        return np.log((J(z) + z*nu - rho)/(1 - rho))/nu 
    def bessel(z):
        return iv(r, z)
    def K1(z):
        zf = Z(K) + z
        kk= 1/8*(2-3*(rho-nu*z)**2/J(z)**2)*nu*nu-1/4*rho*beta*nu/(1-beta)/zf
        return kk
    def K2(z):
        zf = Z(K) + z
        return -beta*(2-beta)/8/(1-beta)**2/(zf**2)
    
    if(beta ==1):
        z = np.log(F/K)/alpha
    else:
        z = Z(F)-Z(K)
    def h(z):
        rho_=np.sqrt(1-rho*rho)
        return 1/2*beta*rho/(1-beta)/J(-Z(K))**2*(nu*Z(K)*np.log(Z(K)*J(z)/Z(F))+(1+rho*nu*Z(K))/rho_*(np.arctan((nu*z-rho)/rho_)+np.arctan(rho/rho_)))
    def hw(z):
        rho_ = np.sqrt(1-rho**2)

        mu0 = rho/rho_+(J(z)-1)/nu/z/rho_
        L = J(z)/nu/Z(K)/rho_
        if L <1:
            L_ = np.sqrt(1-L**2)
            I = 2/L_*(np.arctan((mu0+L)/L_)-np.arctan(L/L_))
            if(1-L)<1.e-2:
                C = mu0/(1+mu0)
                A = (2*mu0-1)/(1+mu0)/6
                B = (1-13*mu0+16*mu0**2)/(1+mu0)**2/120
                c2 = mu0**2*(3+mu0)/6/(1+mu0)**3+8*A*(7/360+(B-A*C*C+C**4/5+(A-C**2/3)/6))
                I = 2*mu0/(1+mu0)+mu0*(3*mu0-1)/2/(1+mu0)**2*(1-L)+c2*(1-L)**2
        else:
            L_ = np.sqrt(L**2-1)
            I = 1/L_*np.log((mu0*(L+L_)+1)/(mu0*(L-L_)+1))
            if(L-1)<1.e-2:
                b4 = mu0**2*(7*mu0**3+35*mu0**2+85*mu0-15)/180/(1+mu0)**5
                d2 = 4*b4+mu0**mu0*(3+mu0)/9/(1+mu0)**2
                I = 2*mu0/(1+mu0)+mu0*(1+5*mu0)/2/(1+mu0)**2*(L-1)+d2*(L-1)**2
        hw_z =-0.5*beta/(1-beta)*rho/rho_*(np.pi-np.arccos((nu*z-rho)/J(z))-np.arccos(rho)-I)
        hh = h(z)
        #hh = -1/12*rho*nu*beta/(1-beta)/(Z(K)+z/2)*z*z
        return hw_z
                                              
    if (beta == 0):
        pdf = 1/(alpha*K**beta)*J(z)**(-1.5)/np.sqrt(2*np.pi*T)*np.sqrt(F**beta/K**beta)*np.exp(-X(z)**2/2/T)
    if beta ==1:
        pdf = 1/(alpha*J(z)**(-1.5)*np.sqrt(2*np.pi*T*F*K))*np.exp(-X(z)**2/2/T)
    else:
        # pdf = 1/(alpha*K**beta)*J(z)**(-1.5)/T*Z(F)**r*Z(K)**(1-r)*bessel(Z(F)*Z(K)/T)*np.exp(-X(z)**2/2/T-Z(F)*Z(K)/T+hw(z)+(K1(z/2))*T)
        pdf = 1/alpha/K**beta*J(z)**(-1.5)/np.sqrt(2*np.pi*T)*np.sqrt(F**beta/K**beta)\
            *np.exp(-X(z)**2/2/T+T*(K1(z/2)+K2(z/2))+hw(z))

    return pdf

def sabr_pdf_bad(F, F0, alpha0, beta, nu, rho0, T):
    rho = rho0
    beta_ = 1-beta
    beta_2 = beta_**2
    rho_ = np.sqrt(1-rho**2)
    F_ = F**beta_

    z_F =  F_/alpha0/beta_
    z_F0 = F0**beta_/alpha0/beta_
    z = z_F0-z_F
    J_z= np.sqrt(1-2*rho*nu*z+nu**2*z**2)
    x_z = 1/nu*np.log((J_z-rho+nu*z)/rho_)
    z0 = z/2
    J_z0= np.sqrt(1-2*rho*nu*z0+nu**2*z0**2)
    k_z0 = 1/8*(2-3*(rho-nu*z0)**2/J_z0**2)*nu*nu-1/4*rho*nu*beta/beta_/(z_F+z0)\
        -beta*(2-beta)/8/beta_**2/(z_F+z0)**2
    mu0 = rho/rho_+(J_z-1)/nu/z/rho_
    L = J_z/nu/z_F/rho_
    if L <1:
        L_ = np.sqrt(1-L**2)
        I = 2/L_*(np.arctan((mu0+L)/L_)-np.arctan(L/L_))
    else:
        L_ = np.sqrt(L**2-1)
        I = 1/L_*np.log((mu0*(L+L_)+1)/(mu0*(L-L_)+1))

    hw_z =-0.5*beta/beta_*rho/rho_*(np.pi-np.arccos((nu*z-rho)/J_z)-np.arccos(rho)-I)
    J_zF= np.sqrt(1+2*rho*nu*z_F+nu**2*z_F**2)
    hw_z = 1/2*beta*rho/beta_/J_zF**2*(nu*z_F*np.log(z_F*J_z/z_F0)+(1+rho*nu*z_F)/rho_*(np.arctan((nu*z-rho)/rho_)+np.arctan(rho/rho_)))
    #hw_z = -1/12*rho*nu*beta/beta_/(z_F+z0)*z*z #hagan
    #hw_z = 1/2*rho*nu*z/2*beta*(1-beta)
    
    pdf = 1/alpha0/F**beta*J_z**(-1.5)/np.sqrt(2*np.pi*T)*np.sqrt(F0**beta/F**beta)\
    *np.exp(-x_z**2/2/T+T*k_z0+hw_z)

    r = 1/2/beta_
    z_t = z_F*z_F0/T
    k_z0 = 1/8*(2-3*(rho-nu*z0)**2/J_z0**2)*nu*nu-1/4*rho*nu*beta/beta_/(z_F+z0)\
        #-beta*(2-beta)/8/beta_**2/(z_F+z0)**2
    # pdf = 1/alpha0/F**beta*J_z**(-1.5)/T*z_F**(1-r)*z_F0**r\
    # *np.exp(-x_z**2/2/T-z_t + hw_z+T*k_z0)*iv(r,z_t)
    pdf = np.max(pdf, 0)  # ensure non-negative values

    return pdf
def sabr_pdf(F, F0, alpha0, beta, nu, rho0, T):
    return sabr_xzz_pdf(F, T, F0, alpha0, beta, rho, nu)

def sabr_pdf_xzz(K, F, alpha, beta, nu, rho, T):
    r = 0.5/(1-beta)
    def Z(f):
        return f**(1-beta)/alpha/(1-beta)
    def J(z):
        return (1 - 2*rho*nu*z + (nu*z)**2)**0.5
    def X(z):
        return np.log((J(z) + z*nu - rho)/(1 - rho))/nu 
    def bessel(z):
        return iv(r, z)
    def K1(z):
        zf = Z(K) + z
        return 1/8*(2-3*(rho-nu*z)**2/J(z)**2)*nu*nu-1/4*rho*beta*nu/(1-beta)/zf
    def K2(z):
        zf = Z(K) + z
        return -beta*(2-beta)/8/(1-beta)**2/(zf**2)
    
    if(beta ==1):
        z = np.log(F/K)/alpha
    else:
        z = Z(F)-Z(K)
    def h(z):
        rho_=np.sqrt(1-rho*rho)
        return 1/2*beta*rho/(1-beta)/J(-Z(K))**2*(nu*Z(K)*np.log(Z(K)*J(z)/Z(F))+(1+rho*nu*Z(K))/rho_*(np.arctan((nu*z-rho)/rho_)+np.arctan(rho/rho_)))
    def hw(z):
        rho_ = np.sqrt(1-rho**2)

        mu0 = rho/rho_+(J(z)-1)/nu/z/rho_
        L = J(z)/nu/Z(K)/rho_
        if L <1:
            L_ = np.sqrt(1-L**2)
            I = 2/L_*(np.arctan((mu0+L)/L_)-np.arctan(L/L_))
            if(1-L)<1.e-2:
                C = mu0/(1+mu0)
                A = (2*mu0-1)/(1+mu0)/6
                B = (1-13*mu0+16*mu0**2)/(1+mu0)**2/120
                c2 = mu0**2*(3+mu0)/6/(1+mu0)**3+8*A*(7/360+(B-A*C*C+C**4/5+(A-C**2/3)/6))
                I = 2*mu0/(1+mu0)+mu0*(3*mu0-1)/2/(1+mu0)**2*(1-L)+c2*(1-L)**2
        else:
            L_ = np.sqrt(L**2-1)
            I = 1/L_*np.log((mu0*(L+L_)+1)/(mu0*(L-L_)+1))
            if(L-1)<1.e-2:
                b4 = mu0**2*(7*mu0**3+35*mu0**2+85*mu0-15)/180/(1+mu0)**5
                d2 = 4*b4+mu0**mu0*(3+mu0)/9/(1+mu0)**2
                I = 2*mu0/(1+mu0)+mu0*(1+5*mu0)/2/(1+mu0)**2*(L-1)+d2*(L-1)**2
                
        hw_z =-0.5*beta/(1-beta)*rho/rho_*(np.pi-np.arccos((nu*z-rho)/J(z))-np.arccos(rho)-I)
        return hw_z
                                              
    if (beta == 0):
        pdf = 1/(alpha*K**beta)*J(z)**(-1.5)/np.sqrt(2*np.pi*T)*np.sqrt(F**beta/K**beta)*np.exp(-X(z)**2/2/T)
    if beta ==1:
        pdf = 1/(alpha*J(z)**(-1.5)*np.sqrt(2*np.pi*T*F*K))*np.exp(-X(z)**2/2/T)
    else:
        pdf = 1/(alpha*K**beta)*J(z)**(-1.5)/T*Z(F)**r*Z(K)**(1-r)*bessel(Z(F)*Z(K)/T)*np.exp(-X(z)**2/2/T-Z(F)*Z(K)/T+hw(z)+(K1(z/2))*T)
    return pdf
from scipy.integrate import quad
def mean_pdf(pdf, *args):
    """
    Integrate the PDF over the range of F.
    """
    maxF = args[0]*15
    return quad(lambda F: F*pdf(F, *args), 0, maxF)[0]

def xzz_call_price(K, F0, alpha0, beta, nu, rho, T,scale):
    def integrand(F):
        return sabr_pdf(F, F0, alpha0, beta, nu, rho, T) *scale* np.maximum(F - K, 0)
    maxF = F0*15
    call_price, _ = quad(integrand, 0, maxF)

    return call_price


def call_price(F_T_regularized, K0):
    payoff = np.maximum(F_T_regularized- K0, 0)
    option_price = np.mean(payoff) 
    return option_price

from scipy.stats import norm
from scipy.optimize import brentq

# --- Black-Scholes and Implied Volatility Functions ---
def black_scholes_call(S, K, T, r, sigma):
    """
    Calculates the Black-Scholes price for a European call option.

    Args:
        S (float): Current stock price.
        K (float): Strike price.
        T (float): Time to maturity in years.
        r (float): Risk-free interest rate.
        sigma (float): Volatility.

    Returns:
        float: Price of the call option.
    """
    if sigma <= 0 or T <= 0:
        return max(0, S - K * np.exp(-r * T))
        
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    return price

def implied_volatility(call_price, S, K, T, r):
    """
    Calculates the implied volatility for a European call option.

    Args:
        call_price (float): The market price of the call option.
        S (float): Current stock price.
        K (float): Strike price.
        T (float): Time to maturity.
        r (float): Risk-free interest rate.

    Returns:
        float: The implied volatility.
    """
    try:
        # Objective function: difference between Black-Scholes price and market price
        def objective_function(sigma):
            return black_scholes_call(S, K, T, r, sigma) - call_price

        # Use Brent's method to find the root (the implied volatility)
        # We provide a reasonable bracket for volatility [0.001, 2.0]
        return brentq(objective_function, 1e-3, 2.0)
    except ValueError:
        # If the solver fails, it may be because the price is outside the no-arbitrage bounds
        return np.nan

if __name__ == '__main__':
    import matplotlib.pyplot as plt
    # Model and Simulation Parameters
    F0 = 1          # Initial forward price
    sigma0 = 0.25        # Initial volatility
    beta = 0.6          # Beta
    nu = 0.3            # Vol-of-vol
    rho = -0.8          # Correlation

    # F0 = 0.0488
    # sigma0 = 0.026        # Initial volatility
    # rho=-0.1
    # beta = 0.5
    T = 2             # Maturity in years
    n_steps = 1000
    dt = T/n_steps
    num_simulations = 100000 # Number of simulations
    r =0
    # Option Parameters

    K = np.array([0.4,.485,.57,.655,.74,.825,.91,.995,1.08,1.165,1.25,1.335,1.42,1.505,1.59,1.675,1.76,1.845,1.93,2.015,2.1])           # Strike price
    K = np.linspace(0.1,4,40)
    # --- Using the Regularized Scheme from the Paper ---
    print("Running simulation with orignal Euler Scheme...")
    S_paths_sim, alpha_paths_sim = simulate_displaced_sabr(
        S0=F0, 
        delta=0, 
        alpha0=sigma0, 
        beta=beta, 
        nu=nu, 
        rho=rho, 
        T=T, 
        num_steps=n_steps, 
        num_paths=num_simulations
    )
    F_T_regularized = S_paths_sim[-1,]
    ivol_mc = []
    for k in K:
        call_price_orignal_eula = call_price(F_T_regularized, k)
        print(f"Call Option Price (original Scheme): {call_price_orignal_eula:.6f}")
        ivol = implied_volatility(call_price_orignal_eula, F0, k, T, r)
        ivol_mc.append(ivol*100)
        print(f"K {k:.4f}, Implied Volatility: {ivol:.4f} (or {ivol*100:.2f}%)\n")

# --- Using the Regularized Scheme from the Paper ---
    # print("Running simulation with Regularized Euler Scheme...")
    # dt = T/n_steps
    # F_T_regularized = sabr_simulation_regularized(F0, sigma0, beta, nu, rho, T, dt, num_simulations)
    # call_price_regularized = call_price(F_T_regularized, K)
    # print(f"Call Option Price (Regularized Scheme): {call_price_regularized:.4f}\n")
    # iv = implied_volatility(call_price_regularized, F0, K, T, r)
    # print(f"Implied Volatility: {iv:.4f} (or {iv*100:.2f}%)")

    # --- Using the Standard Log-Euler Scheme ---
    # print("Running simulation with Log-Euler Scheme...")
    # F_T_log_euler = sabr_simulation_log_euler(F0, sigma0, beta, nu, rho, T, dt, num_simulations)
    # for k in K:
    #     call_price_log_euler = call_price(F_T_log_euler, k)
    #     print(f"K{k:.4f} Call Option Price (Log-Euler Scheme): {call_price_log_euler:.4f}")
    #     ivol = implied_volatility(call_price_log_euler, F0, k, T, r)
    #     print(f"Implied Volatility: {ivol:.4f} (or {ivol*100:.2f}%)\n")

    #--- Using the xzz Scheme ---
    print("Running xzz Scheme...")

    f_points = np.linspace(1.e-10,F0*20,n_steps)
    df = F0*20/n_steps
    p_points=[]
    p_points_xzz=[]
    scale = F0/mean_pdf(sabr_pdf, F0, sigma0, beta, nu, rho, T)
    scale_xzz = F0/mean_pdf(sabr_pdf_xzz, F0, sigma0, beta, nu, rho, T)
    for f in f_points[1:]:
        p_points.append(f*sabr_pdf(f,F0,sigma0,beta,nu,rho,T)*df)
        p_points_xzz.append(f*sabr_pdf_xzz(f,F0,sigma0,beta,nu,rho,T)*df)
    plt.figure(figsize=(12, 5))
    plt.plot(f_points[3:], p_points[2:]) # Plot first 10 paths for S_t
    plt.plot(f_points[3:], p_points_xzz[2:]) # Plot first 10 paths for S_t
    plt.show()

    scale = F0/mean_pdf(sabr_pdf, F0, sigma0, beta, nu, rho, T)

    ivol_xzz = []
    for k in K:
        call_price0 = xzz_call_price(k,F0,sigma0,beta,nu,rho,T,scale)
        print(f"K{k:.4f} Call Option Price (xzz Scheme): {call_price0:.4f}")
        ivol = implied_volatility(call_price0, F0, k, T, r)
        ivol_xzz.append(ivol*100)
        print(f"Implied Volatility: {ivol:.4f} (or {ivol*100:.2f}%)\n")

    ivol_hg = []
    for k in K:
        ivol = sabr_implied_vol(k, T=T, F=F0, alpha=sigma0, beta=beta, rho=rho, nu=nu)
        ivol_hg.append(ivol*100)
        print(f"K{k:.4f} Hagan Implied Volatility: {ivol:.4f} (or {ivol*100:.2f}%)\n")

    plt.figure(figsize=(10, 6))
    plt.plot(K, ivol_mc, marker='o', label='MC Log-Euler')
    plt.plot(K, ivol_xzz, marker='x', label='XZZ PDF')
    plt.plot(K, ivol_hg, marker='s', label='Hagan')
    plt.xlabel('Strike (K)')
    plt.ylabel('Implied Volatility (%)')
    plt.title('Implied Volatility Smile: MC Log-Euler vs XZZ PDF vs Hagan')
    plt.legend()
    plt.grid(True)
    plt.show()

    #For plotting (requires matplotlib)
    
    # time_points = np.linspace(0, T, n_steps + 1)

    # plt.figure(figsize=(12, 5))
    # plt.subplot(1, 2, 1)
    # plt.plot(time_points, S_paths_sim[:, :10]) # Plot first 10 paths for S_t
    # plt.title('Simulated S_t Paths (Log-Euler)')
    # plt.xlabel('Time (Years)')
    # plt.ylabel('S_t')

    # plt.subplot(1, 2, 2)
    # plt.plot(time_points, alpha_paths_sim[:, :10]) # Plot first 10 paths for alpha_t
    # plt.title('Simulated alpha_t Paths (Log-Euler)')
    # plt.xlabel('Time (Years)')
    # plt.ylabel('alpha_t')
    # plt.tight_layout()
    # plt.show()
